# NonLinear Programming

##### Funzione

$$
f(x,y,z) = -(2x^2+7x+5y+z) \\
$$

##### Vincoli

$$
    -x-2y-3z >= -8 \\
    +2x-3y-z >= -6 \\
    x >= 0 \\
    y >= 0 \\
    z >= 0 
$$

## Scipy

In [1]:
import numpy as np
from scipy.optimize import least_squares, minimize

In [2]:
def fn_target(x):
    
    z = x[2]
    y = x[1]
    x = x[0]
    
    return -1*(2*𝑥**2+7*𝑥+5*𝑦+𝑧)

In [3]:
def constraint_1(x):
    
    z = x[2]
    y = x[1]
    x = x[0]
    
    return -𝑥 -2*𝑦 -3*𝑧 - 8

In [4]:
def constraint_2(x):
    
    z = x[2]
    y = x[1]
    x = x[0]
    
    return +2*𝑥 -3*𝑦 -𝑧 - 6

In [5]:
np.random.seed(12)
x0 = np.random.randint(low=[100],size=3)
x0

array([75, 27,  6])

In [6]:
print(fn_target(x0))

-11916


In [7]:
b = (0.0,None)
bnds = (b,b,b)

In [8]:
con1 = {'type':'ineq', 'fun':constraint_1}
con2 = {'type':'ineq', 'fun':constraint_2}
cons = [con1, con2]

In [9]:
solution = minimize(
    fun = fn_target,
    x0 = x0,
    args=(),
    method='SLSQP',
    jac=None,
    hess=None,
    hessp=None,
    bounds=bnds,
    constraints=cons,
    tol=None,
    callback=None,
    options={'maxiter':100},
)

In [10]:
if solution.success:
    print(solution)

In [11]:
fn_target(solution.x)

-44247.000002906025

## GEKKO

In [12]:
import numpy as np
from gekko import GEKKO

In [13]:
m = GEKKO(remote=False)

In [14]:
x,y,z = m.Array(m.Var, 3, lb=0, ub=None, integer=False)

In [15]:
m.Minimize(-1*(2*𝑥**2+7*𝑥+5*𝑦+𝑧))

In [16]:
m.Equation(  -𝑥 -2*𝑦 -3*𝑧 >= -8)
m.Equation(+2*𝑥 -3*𝑦   -𝑧 >= -6)

In [17]:
m.solve(disp=False)

In [18]:
print(f'Soluzione ottimale: x = {x.value[0]:.2f} , y = {y.value[0]:.2f}, z = {z.value[0]:.2f}')
print(f'Valore massimo: {-m.options.OBJFCNVAL:.2f}')

Soluzione ottimale: x = 8.00 , y = 0.00, z = 0.00
Valore massimo: 184.00


In [19]:
from hyperopt import fmin, tpe, hp
from hyperopt import STATUS_OK, STATUS_FAIL

In [20]:
# hyperparameters space
space = {
    
    'x':hp.quniform('x', 0, 100, 3),
    'y':hp.quniform('y', 0, 100, 3),
    'z':hp.quniform('z', 0, 100, 3),
}

In [21]:
def objective(params):
    
    m = GEKKO(remote=False)
    
    variables = m.Array(m.Var, 3, lb=0, ub=None, integer=False)
    x,y,z = variables
    
    x.value = params['x']
    y.value = params['y']
    z.value = params['z']

    m.Minimize(-1*(2*𝑥**2+7*𝑥+5*𝑦+𝑧))
    
    m.Equations([  -𝑥 -2*𝑦 -3*𝑧 >= -8,
               +2*𝑥 -3*𝑦   -𝑧 >= -6])
        
    m.solve(disp=False, debug=False)
    
    obj = m.options.OBJFCNVAL
    
    if m.options.APPSTATUS==1:
        s=STATUS_OK
    else:
        s=STATUS_FAIL
        
    m.cleanup()
    
    return {'loss':obj, 'status':s, 'x':variables}

In [22]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=50)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 20.15trial/s, best loss: -184.0]


In [23]:
print(best)

{'x': 90.0, 'y': 3.0, 'z': 24.0}


In [24]:
sol = objective(best)

In [25]:
print(sol)

{'loss': -184.0, 'status': 'ok', 'x': array([[8.0], [0.0], [0.0]], dtype=object)}


# TensorFlow

In [26]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [27]:
import tensorflow as tf

In [28]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "non available")

Version:  2.8.0
Eager mode:  True
GPU is available


In [55]:
tf.random.set_seed(100)

# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

variable_initializer = tf.random_uniform_initializer(minval=10, maxval=20)

variable_initializer(shape=[1,])

x = tf.Variable(variable_initializer(shape=[1,]), dtype=tf.float32)
y = tf.Variable(variable_initializer(shape=[1,]), dtype=tf.float32)
z = tf.Variable(variable_initializer(shape=[1,]), dtype=tf.float32)

fn_target = lambda: -1*(2*𝑥**2 +7*𝑥 +5*𝑦 +𝑧)

i = 0

with tf.GradientTape() as g:
    g.watch(x)
    g.watch(y)
    g.watch(z)

#     t = fn_target()
#     gradients = g.gradient(t, sources=[x,y,z])
#     sdg.apply_gradients(zip(gradients, [x,y]))

    while abs(fn_target() - 0) >= 0.01:
        i = i + 1
        n = optimizer.minimize(fn_target, var_list=[x,y,z])
        
        if i % 100 == 0:
            print(f"iter n.{i}:\n    target={fn_target()[0]} \n    x={x.numpy()[0]} y={y.numpy()[0]} z={z.numpy()[0]}\n")

        if i == 500:
            break

iter n.100:
    target=-901.630126953125 
    x=18.681833267211914 y=12.152972221374512 z=12.070684432983398

iter n.200:
    target=-994.0947875976562 
    x=19.7137393951416 y=13.152995109558105 z=13.070707321166992

iter n.300:
    target=-1092.8414306640625 
    x=20.768112182617188 y=14.1530179977417 z=14.070730209350586

iter n.400:
    target=-1198.0001220703125 
    x=21.84331512451172 y=15.153040885925293 z=15.07075309753418

iter n.500:
    target=-1309.674072265625 
    x=22.937679290771484 y=16.15306282043457 z=16.070775985717773



In [30]:
x.numpy()

array([22.93768], dtype=float32)

In [31]:
y.numpy()

array([16.153063], dtype=float32)

In [32]:
z.numpy()

array([16.070776], dtype=float32)

In [33]:
fn_target()

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1309.6741], dtype=float32)>

In [34]:
constraint_1([x.numpy(),y.numpy(),z.numpy()])

array([-111.45613], dtype=float32)

In [35]:
constraint_2([x.numpy(),y.numpy(),z.numpy()])

array([-24.654608], dtype=float32)